**I need to make major improvements in HUD**
1. Change its Menu and lists to an object based aproach
    - With this improvement it will be esier to create as many menues in the future
2. Add a menu called experiment list that consists of `Default experiment`, and other experiments I wish to add

Firstly start by importing all the necessary libraries and classes

In [58]:
%pylab inline
import pygame
import pygame.time
from Constants import *
from Snake import *
from HUD import HUD
from collections import deque
from Controls import Controls

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


c:\Users\makho\miniconda3\envs\mysnakegame_env_py312\Lib\site-packages\IPython\core\magics\pylab.py:166: UserWarning: pylab import has clobbered these variables: ['Arrow', 'copy', 'random', 'mean']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


The bellow class called `Game` is a class which is responsible for starting and running the Snake game it starts with an initial state `MenuState` which calls on `HUD.py` in order to render the menu and its items

In [59]:
class Game:
    """
    Main game class that handles the game loop, state management, and overall game logic.
    """
    def __init__(self):
        """
        Initialize the game with all necessary components and settings.
        """
        # Initialization
        pygame.init()
        self.clock = pygame.time.Clock()
        self.window = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
        pygame.display.set_caption("Snake Game")

        # Game state and settings
        self.game_update_rate = LEVEL_1
        self.menu_update_rate = MENUE_RATE
        
        # Game components
        self.game_hud = HUD(self.window)
        self.controls = Controls()
        
        # Create background
        self.background = self.create_background()

        self.running = True

        self.previous_state = None
        from GameUI import MenuState
        self.state = MenuState()
        self.state.enter(self)

    def run(self):
        """
        Main game loop that handles events, updates game state, and draws the game.
        """
        while self.running:
            self.handle_events()
            self.update()
            self.draw()
            
        self.cleanup()

    def handle_events(self):
        """
        Handle all game events including user input and game state changes.
        """
        events = pygame.event.get()
        self.controls.update(events)
        self.state.handle_events(self, events)
        for event in events:
            if event.type == pygame.QUIT:
                self.running = False

    def draw(self):
        """
        Draw the game screen based on the current game state.
        """
        self.window.blit(self.background, (0, 0))
        self.state.draw(self)
        pygame.display.flip()

    def update(self):
        """
        Update game logic, including environment, snake movement, and game over conditions.
        """
        self.state.update(self)

    def cleanup(self):
        """
        Perform cleanup operations before exiting the game.
        """
        pygame.quit()

    def create_background(self):
        """
        Create and return the game background surface.

        Returns:
            pygame.Surface: The created background surface.
        """
        background = pygame.Surface((SCREEN_WIDTH, SCREEN_HEIGHT))
        for x in range(0, SCREEN_WIDTH, CELL_SIZE):
            for y in range(0, SCREEN_HEIGHT, CELL_SIZE):
                color = (60, 60, 60) if (x + y) // CELL_SIZE % 2 == 0 else (40, 40, 40)
                pygame.draw.rect(background, color, (x, y, CELL_SIZE, CELL_SIZE))
        return background


As you have seen, the game starts, showing a detailed menu with animations and navigation ability, now the problem with this is that everytime when I create a sub-menu I need to perform a tideous job of creating multiple functions and systems before I can have a working menu.

**Step 1:** Select one spercific menu system and list all its parts and iterations.

In that case I will pick the Main menu.
- It has: 
    ```python
    # list of all items that make the menu
    self.menu_items
    ```
    ```python 
    # Index of the selected item
    self.selected_menu_item
    ```
    ```python
    # Renders and draws evrything in the list on the display
    def draw_menu(self):
    ``` 
    ```python
    # It changes the value of self.selected_menu_item by direction (-1 or 1)
    def navigate_menu(self, direction):
    ```
    ```python
    # Should be Selected menu item
    # it returns the string of the selected item
    def select_menu_item(self):
    ```
- These are its helper functions:
    ```python
    def _create_gradient_background(self):
    ```
    ```python
    def _create_menu_background(self):
    ```
    ```python
    def _create_button_background(self):
    ```
    ```python
    def _interpolate_color(color1, color2, factor):
    ```
    ```python
    def _render_text(self, text, font, color):
    ```
    ```python
    def _draw_centered_text(self, text_surface, y_offset=0):
    ```

Let us give the main menu a test run
> *Do not press enter on any option but **Quit** !*

In [60]:
game = Game()
game.run()

No joystick connected.


The drawing, updating and handling of events in the menu are handled by the `MenuState`

In [61]:
from GameUI import MenuState
%psource MenuState

class MenuState(GameState):
    def enter(self, game: Game):
        pass

    def handle_events(self, game: Game, events: list[pygame.event.Event]):

        for event in events:
            if event.type == pygame.KEYDOWN or event.type == pygame.JOYBUTTONDOWN:
                self._handle_actions(game)
                

    def _handle_actions(self, game: Game):
        if game.controls.is_just_pressed('UP'):
            game.game_hud.navigate_menu(-1)
        elif game.controls.is_just_pressed('DOWN'):
            game.game_hud.navigate_menu(1)
        elif game.controls.is_just_pressed('SELECT'):
            self._handle_menu_select(game)

    
    def update(self, game: Game):
        game.clock.tick(game.menu_update_rate)
    
    def draw(self, game: Game):
        game.game_hud.draw_menu()
    
    def exit(self, game: Game):
        pass

    def _handle_menu_select(self, game: Game):
        selected_item = game.game_hud.select_menu_item()
        if selected_item == "Quit":


The draw function calls `game.game_hud.draw_menu()` then the events function calls actions and each action navigates the menu `game.game_hud.navigate_menu(-1)` (**-1** navigates up) from the HUD class.

Another thing about the Menu class is that it has a menu select handler `def _handle_menu_select(self, game: Game):` thus when I click on **Enter** I can change my state to the corresponding state

Bellow I have made changes to the game class, I will be calling all raw functions

In [62]:
class UpdatedGame:
    def __init__(self):
        # Initialize pygame, create a display and set the caption of the display
        pygame.init()
        self.clock = pygame.time.Clock()
        self.display = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
        pygame.display.set_caption("Snake Game")

        # Is the game running and how fast does it run
        self.running = True
        self.menu_update_rate = MENUE_RATE

        #Settup the display hud and controls
        self.game_hud = HUD(self.display)
        self.controls = Controls()

    def run(self):
        while self.running:
            self.handle_events()
            self.update()
            self.draw()

        pygame.quit()

    def handle_events(self):
        events = pygame.event.get()
        self.controls.update(events)
        # Directly update the menu
        for event in events:
            if event.type == pygame.KEYDOWN:
                if self.controls.is_just_pressed(self.controls.UP):
                    self.game_hud.navigate_menu(-1)
                elif self.controls.is_just_pressed(self.controls.DOWN):
                    self.game_hud.navigate_menu(1)
                elif self.controls.is_just_pressed(self.controls.SELECT):
                    pass

        for event in events:
            if event.type == pygame.QUIT:
                self.running = False

    def update(self):
        self.clock.tick(game.menu_update_rate)

    def draw(self):
        # Directly draw the menu
        self.game_hud.draw_menu()
        pygame.display.flip()

Let's test it out

In [63]:
game = UpdatedGame()
game.run()

No joystick connected.


Now we know what is necesary to achieve a working menu
- Based on the above implementation I only need to implement ***drawing*** and ***navigating*** of the menu.
- Selecting items (by clicking **Enter**) has not been handled

Now let us look at the game HUD

In [70]:
%psource HUD

No source found for HUD


And as you can see it is fairly complicated. The reason for this is due to redundences a lot of repeated code which is what I want to remove

Bellow I will create a new HUD wich is more modular

In [65]:
class HUD:
    def __init__(self, window: pygame.Surface):
        """
        Initialize the HUD with necessary components.

        Args:
            window (pygame.Surface): The main game window surface for rendering.
        """
        self.window = window
        self.fonts = self._load_fonts()
        self.score = 0 
        self.menu_lists = {
            "main_menu": ["New Game", "Continue", "Level", "AI", "High Scores", "Quit"],
            "game_level_menu": ["EASY", "MEDIUM", "HARD", "VERY HARD"],
            "ai_vision_menu": ["AI Vision", "AI Play", "AI Experiment"],
            "ai_play_menu": ["Breadth First", "Depth First", "A Star", "Greedy"],
            "high_scores_menu": [[0] * MAX_HIGH_SCORES for _ in range(4)]
        }
        self.selected_menu_item = [0] * len(self.menu_lists)
        self.current_level = "MEDIUM"
        self.animations = {}
        self.load_assets()

    # --- Initialization Methods ---
    def _load_fonts(self):
        """
        Load and return a dictionary of pygame Font objects for different text sizes.

        Returns:
            dict: A dictionary of pygame Font objects.
        """
        return {
            'small': pygame.font.Font(None, 32),
            'medium': pygame.font.Font(None, 48),
            'large': pygame.font.Font(None, 72),
            'title': pygame.font.Font(None, 96)
        }

    def load_assets(self):
        """
        Load and create necessary graphical assets for the HUD.
        """
        self.background = self._create_gradient_background()
        self.menu_bg = self._create_menu_background()
        self.button_bg = self._create_button_background()

    # --- Asset Creation Methods ---
    def _create_gradient_background(self):
        """
        Create and return a gradient background surface.

        Returns:
            pygame.Surface: A surface with a vertical gradient from cyan to magenta.
        """
        background = pygame.Surface((SCREEN_WIDTH, SCREEN_HEIGHT))
        for y in range(SCREEN_HEIGHT):
            color = self._interpolate_color(BRIGHT_CYAN, BRIGHT_MAGENTA, y / SCREEN_HEIGHT)
            pygame.draw.line(background, color, (0, y), (SCREEN_WIDTH, y))
        return background

    def _create_menu_background(self):
        """
        Create and return a semi-transparent background surface for menus.

        Returns:
            pygame.Surface: A semi-transparent surface for menu backgrounds.
        """
        menu_bg = pygame.Surface((SCREEN_WIDTH * 0.8, SCREEN_HEIGHT * 0.8), pygame.SRCALPHA)
        pygame.draw.rect(menu_bg, (0, 0, 0, 180), menu_bg.get_rect(), border_radius=20)
        return menu_bg

    def _create_button_background(self):
        """
        Create and return a semi-transparent background surface for buttons.

        Returns:
            pygame.Surface: A semi-transparent surface for button backgrounds.
        """
        button_bg = pygame.Surface((300, 60), pygame.SRCALPHA)
        pygame.draw.rect(button_bg, (255, 255, 255, 50), button_bg.get_rect(), border_radius=10)
        return button_bg
    
    # --- Utility Methods ---
    @staticmethod
    def _interpolate_color(color1, color2, factor):
        """
        Interpolate between two colors.

        Args:
            color1 (tuple): The first color (R, G, B).
            color2 (tuple): The second color (R, G, B).
            factor (float): Interpolation factor between 0 and 1.

        Returns:
            tuple: The interpolated color (R, G, B).
        """
        return tuple(int(color1[i] + (color2[i] - color1[i]) * factor) for i in range(3))
    
    def _render_text(self, text, font, color):
        """
        Render text using the specified font and color.

        Args:
            text (str): The text to render.
            font (pygame.font.Font): The font to use for rendering.
            color (tuple): The color of the text (R, G, B).

        Returns:
            pygame.Surface: A surface containing the rendered text.
        """
        return font.render(text, True, color)

    def _draw_centered_text(self, text_surface, y_offset=0):
        """
        Draw text centered on the screen with a vertical offset.

        Args:
            text_surface (pygame.Surface): The rendered text surface to draw.
            y_offset (int, optional): Vertical offset from the center of the screen. Defaults to 0.
        """
        text_rect = text_surface.get_rect(center=(SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 + y_offset))
        self.window.blit(text_surface, text_rect)
    

    # --- Drawing Methods ---
    def draw_menu(self):
        """
        Draw the main menu screen.
        """
        ## Renders the tittle text and draws it on screen
        title_text = self._render_text("Snake Game", self.fonts['title'], BRIGHT_GOLD)
        self._draw_centered_text(title_text, -250)

        # Renders the backgrounds on the window
        self.window.blit(self.background, (0, 0))
        self.window.blit(self.menu_bg, (SCREEN_WIDTH * 0.1, SCREEN_HEIGHT * 0.1))

        for index, item in enumerate(self.menu_lists["main_menu"]):
            ## Button Background
            # Slelect the y coordinate starting from the top for the bautton backgrouund
            # Render the background of the button on the window
            button_y = SCREEN_HEIGHT // 2 + index * 80 - 190
            self.window.blit(self.button_bg, (SCREEN_WIDTH // 2 - 150, button_y))
            
            ## Button Text
            # Select the color of the text based on thether it is selected
            color = BRIGHT_GOLD if index == self.selected_menu_item[0] else WHITE
            # Render the menu item with a large font
            menu_text = self._render_text(item, self.fonts['large'], color)
            # Change the center of the the text to the position and render it to the window
            text_rect = menu_text.get_rect(center=(SCREEN_WIDTH // 2, button_y + 30))
            self.window.blit(menu_text, text_rect)

            ## Button Animation
            # If the item is slelcted animate it
            if index == self.selected_menu_item[0]:
                # Store (index = 0) or obtain animation values for each menu item
                animation = self.animations.get(f'menu_{index}', 0)
                # Compute a new value for animation that lies between [0, 2pi)
                animation = (animation + 0.1) % (2 * math.pi)
                # Update the animation value in its index position
                self.animations[f'menu_{index}'] = animation
                
                # Compute the glow factor [0, 1] using the animation angle [0, 2pi)
                glow_factor = (math.sin(animation) + 1) / 2
                # Determine the glow color, shoud it be mostly BRIGHT_GOLD or WHITE (determined by glow factor)
                glow_color = self._interpolate_color(BRIGHT_GOLD, WHITE, glow_factor)
                # Create a transparent surface larger than the button surface (10 x 10)
                glow_surf = pygame.Surface((310, 70), pygame.SRCALPHA)
                # Draw the surface as a rectangle with a smaller border radius radius (15/20)
                pygame.draw.rect(glow_surf, (*glow_color, 100), glow_surf.get_rect(), border_radius=15)
                # Render the surface on the window
                self.window.blit(glow_surf, (SCREEN_WIDTH // 2 - 155, button_y - 5))


The above code is not yet that modulor but explains what happens in detail
- There are 4 backgrounds
    - Screen background
    - Menu background
    - Button background
    - Selected item background `rendered only on selected item`
    
    *These 3 backgrounds all are rendered on surfaces where the menu and button backgrounds are transparent pygame.SRCALPHA*

- The menu is composed of 3 texts
    - Tittle text
    - Item text
    - Selected item text

    *Selected item text has a special color*

Nocie that it all boils down to two objects
1. Menu Item
2. Menu

In [66]:
class MenuButton:
    def __init__(self, text, font, color=WHITE, selected_color=BRIGHT_GOLD):
        self.menu_button_text = self._render_text(text, font, color)
        self.menu_button_selected_text = self._render_text(text, font, selected_color)
        self.button_background = self._create_button_background()
        self.font = font
        self.animation = 0

    def _render_text(self, text, font, color):
        return font.render(text, True, color)
    
    def _create_button_background(self):
        button_bg = pygame.Surface((300, 60), pygame.SRCALPHA)
        pygame.draw.rect(button_bg, (255, 255, 255, 50), button_bg.get_rect(), border_radius=10)
        return button_bg 
    
    @staticmethod
    def _interpolate_color(color1, color2, factor):
        return tuple(int(color1[i] + (color2[i] - color1[i]) * factor) for i in range(3))
    
    def draw(self, window, index, selected=False):
        ## Button Background
            # Slelect the y coordinate starting from the top for the bautton backgrouund
            # Render the background of the button on the window
            button_y = SCREEN_HEIGHT // 2 + index * 80 - 190
            window.blit(self.button_background, (SCREEN_WIDTH // 2 - 150, button_y))
            
            ## Button Text
            # Select the color of the text based on thether it is selected
            color = BRIGHT_GOLD if selected else WHITE
            # Change the center of the the text to the position and render it to the window
            text_rect = self.menu_button_text.get_rect(center=(SCREEN_WIDTH // 2, button_y + 30))
            window.blit(self.menu_button_text, text_rect)

            ## Button Animation
            # If the item is slelcted animate it
            if selected:
                # Compute and update a new value for animation that lies between [0, 2pi)
                self.animation = (self.animation + 0.1) % (2 * math.pi)
                
                # Compute the glow factor [0, 1] using the animation angle [0, 2pi)
                glow_factor = (math.sin(self.animation) + 1) / 2
                # Determine the glow color, shoud it be mostly BRIGHT_GOLD or WHITE (determined by glow factor)
                glow_color = self._interpolate_color(BRIGHT_GOLD, WHITE, glow_factor)
                # Create a transparent surface larger than the button surface (10 x 10)
                glow_surf = pygame.Surface((310, 70), pygame.SRCALPHA)
                # Draw the surface as a rectangle with a smaller border radius radius (15/20)
                pygame.draw.rect(glow_surf, (*glow_color, 100), glow_surf.get_rect(), border_radius=15)
                # Render the surface on the window
                window.blit(glow_surf, (SCREEN_WIDTH // 2 - 155, button_y - 5))
    
class Menu:
    def __init__(self, window, items, tittle_text):
        self.window = window
        self.items = items
        self.font = self._load_fonts()
        self.tittle = MenuButton(tittle_text, self.font['title'], BRIGHT_GOLD)
        self.selected_menu_item = 0
        self.menu_buttons = [MenuButton(item_text, self.font['large']) for index, item_text in enumerate(items)]

    def _load_fonts(self):
        return {
            'small': pygame.font.Font(None, 32),
            'medium': pygame.font.Font(None, 48),
            'large': pygame.font.Font(None, 72),
            'title': pygame.font.Font(None, 96)
        }
    
    @staticmethod
    def _interpolate_color(color1, color2, factor):
        return tuple(int(color1[i] + (color2[i] - color1[i]) * factor) for i in range(3))

    def _create_gradient_background(self):
        background = pygame.Surface((SCREEN_WIDTH, SCREEN_HEIGHT))
        for y in range(SCREEN_HEIGHT):
            color = self._interpolate_color(BRIGHT_CYAN, BRIGHT_MAGENTA, y / SCREEN_HEIGHT)
            pygame.draw.line(background, color, (0, y), (SCREEN_WIDTH, y))
        return background

    def _create_menu_background(self):
        menu_bg = pygame.Surface((SCREEN_WIDTH * 0.8, SCREEN_HEIGHT * 0.8), pygame.SRCALPHA)
        pygame.draw.rect(menu_bg, (0, 0, 0, 180), menu_bg.get_rect(), border_radius=20)
        return menu_bg
    
    def _draw_centered_text(self, text_surface, y_offset=0):
        text_rect = text_surface.get_rect(center=(SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 + y_offset))
        self.window.blit(text_surface, text_rect)
    
    def draw(self):
        # Create the backgrounds and render them on screen
        surface_background = self._create_gradient_background()
        menu_background = self._create_menu_background()
        self.window.blit(surface_background, (0, 0))
        self.window.blit(menu_background, (SCREEN_WIDTH * 0.1, SCREEN_HEIGHT * 0.1))

        # Render the title
        self._draw_centered_text(self.tittle.menu_button_text, -250)

        #Iterate through each button and render them on screen
        for index, button in enumerate(self.menu_buttons):
            if self.selected_menu_item == index:
                button.draw(self.window, index, True)
            else: button.draw(self.window, index)

    def navigate_menu(self, direction):
        self.selected_menu_item = (self.selected_menu_item + direction) % len(self.items)

Now that is done I now modify the HUD class accordingly

In [67]:
class HUD:
    def __init__(self, window: pygame.Surface):
        """
        Initialize the HUD with necessary components.

        Args:
            window (pygame.Surface): The main game window surface for rendering.
        """
        self.window = window
        self.score = 0 
        self.menu_lists = {
            "main_menu": Menu(self.window, ["New Game", "Continue", "Level", "AI", "High Scores", "Quit"], "Snake Game"),
            "game_level_menu": Menu(self.window, ["EASY", "MEDIUM", "HARD", "VERY HARD"], "Snake Game"),
            "ai_vision_menu": Menu(self.window, ["AI Vision", "AI Play", "AI Experiment"], "Snake Game"),
            "ai_play_menu": Menu(self.window, ["Breadth First", "Depth First", "A Star", "Greedy"], "Snake Game"),
            "high_scores_menu": [[0] * MAX_HIGH_SCORES for _ in range(4)]
        }
        self.current_menu = self.menu_lists['main_menu']

    def draw_menu(self):
        self.current_menu.draw()

    def navigate_menu(self, dir):
        self.current_menu.navigate_menu(dir)

That was a bit too comprehensive but I think it is done

Now to test it out bellow, I will firstly create a specialized game class

In [68]:
class UpdatedGame:
    def __init__(self):
        # Initialize pygame, create a display and set the caption of the display
        pygame.init()
        self.clock = pygame.time.Clock()
        self.display = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
        pygame.display.set_caption("Snake Game")

        # Is the game running and how fast does it run
        self.running = True
        self.menu_update_rate = MENUE_RATE

        #Settup the display hud and controls
        self.game_hud = HUD(self.display)
        self.controls = Controls()

    def run(self):
        while self.running:
            self.handle_events()
            self.update()
            self.draw()

        pygame.quit()

    def handle_events(self):
        events = pygame.event.get()
        self.controls.update(events)
        # Directly update the menu
        for event in events:
            if event.type == pygame.KEYDOWN:
                if self.controls.is_just_pressed(self.controls.UP):
                    self.game_hud.navigate_menu(-1)
                elif self.controls.is_just_pressed(self.controls.DOWN):
                    self.game_hud.navigate_menu(1)
                elif self.controls.is_just_pressed(self.controls.SELECT):
                    pass

        for event in events:
            if event.type == pygame.QUIT:
                self.running = False

    def update(self):
        self.clock.tick(game.menu_update_rate)

    def draw(self):
        # Directly draw the menu
        self.game_hud.draw_menu()
        pygame.display.flip()

In [69]:
game = UpdatedGame()
game.run()

No joystick connected.
